In [1]:
import pandas as pd
import numpy as np
import astropy.units as u
import sunpy.map
from AntennaUtils import *  

# Imprimimos estadísticas resumidas del DataFrame final
pd.set_option('display.float_format', '{:.10f}'.format)

e:\CODING\RT-32-SunImage\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# RT32 location (Ventspils, Latvia)
rt32_antenna = RT32()
rt32_antenna.set_location(latitude=57.5535171694, longitude=21.8545525000, elevation=20)

# Define constants
year = 2024
month = 4
day = 25
hour_start = 8
minute_start = 55

temperature = u.Quantity(20.0, unit=u.deg_C)
pressure = u.Quantity(1013.25, unit=u.hPa)
relative_humidity = u.Quantity(60.0, unit=u.percent)
obswl =u.Quantity(50000, unit=u.nm) 

weather = Weather(temperature, pressure, relative_humidity, obswl)

observation = SpiralSunObservation(weather,rt32_antenna , year , month , day , hour_start , minute_start)


print(observation.t_scan)

660.0


In [3]:
#fit_file_path = "lnsp4_241018_163807_241018_163855.fit"

#observation.createImages(fit_file_path)

In [4]:
#CONSTANTS
hdu_number = 1  # Number of the extension containing the binary table
path = ''
fit_file_path = "lnsp4_241018_163807_241018_163855.fit"

#DATA CALCULATION:
az_anten, el_anten , az_sun , el_sun , xx1 , yy1, utc = observation.calculatePositions()
observation.generateFile(path, az_anten , el_anten , utc)  

print('-------------------------------------------------------------')
print('Start processing the FITS file:', fit_file_path)
# Converts the binary table to a Pandas DataFrame
data_df = bintable_to_pandas(fit_file_path, hdu_number)        

print('Processing the data...')
band_data_dfs = processData(data_df)

sunPositionDf = pd.DataFrame({'UTC': utc,'SunX': xx1, 'SunY': yy1  })

print('Getting final processed data...')
processed_dfs = getFinalProcessedData(observation , sunPositionDf,band_data_dfs)

print('Processing heliocentric coordinates...')
band_processed_helio_dfs = process_all_heliocentric_coordinates(processed_dfs, observation )

# Define the target directory
directory = f'{observation.year}-{observation.month}-{observation.day}T{observation.hour_start}_{observation.minute_start}_00'

# Create the directory if it doesn't exist
if not os.path.exists(directory):
    os.makedirs(directory)

bands = ['4.07GHZ', '6.42GHZ', '8.40GHZ', '9.80GHZ', '11.90GHZ']


        

-------------------------------------------------------------
Saved:  sun_scan_240425_0855.ptf    3300   points
-------------------------------------------------------------
Start processing the FITS file: lnsp4_241018_163807_241018_163855.fit
Processing the data...
Getting final processed data...


e:\CODING\RT-32-SunImage\AntennaUtils.py:609: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_of_df[f'RCP_{band}'] = (rest_of_df[f'RCP_{band}'] - sky_means[0]) / (sun_centre_means[0] - sky_means[0])
e:\CODING\RT-32-SunImage\AntennaUtils.py:610: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rest_of_df[f'LCP_{band}'] = (rest_of_df[f'LCP_{band}'] - sky_means[0]) / (sun_centre_means[0] - sky_means[0])
e:\CODING\RT-32-SunImage\AntennaUtils.py:612: SettingWithCopyWarning: 
A value is trying to be set on a copy

Processing heliocentric coordinates...


e:\CODING\RT-32-SunImage\AntennaUtils.py:706: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  band_df['az_anten'] = az_anten
e:\CODING\RT-32-SunImage\AntennaUtils.py:707: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  band_df['el_anten'] = el_anten
e:\CODING\RT-32-SunImage\AntennaUtils.py:727: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [45]:
print('Creating solar maps...')
for band in bands:
    SunX = band_processed_helio_dfs[band]['tx_helio_anten']
    SunY = band_processed_helio_dfs[band]['ty_helio_anten']
    STOKE_I = band_processed_helio_dfs[band][f'STOKE_I_{band}'].values
    STOKE_V = band_processed_helio_dfs[band][f'STOKE_V_{band}'].values

    # Define the grid covering the helioprojective coordinate space
    tx_min, tx_max = -1200, 1200
    ty_min, ty_max = -1200, 1200
    grid_step = 10  # Adjust as needed

    # Create a grid
    tx, ty = np.meshgrid(np.arange(tx_min, tx_max, grid_step),
                        np.arange(ty_min, ty_max, grid_step))

    # Interpolate power values for each point on the grid using Rbf
    rbf = Rbf(SunX, SunY, STOKE_I, function='linear')
    interp_power_STOKE_I = rbf(tx, ty)

    # Create a grid
    tx, ty = np.meshgrid(np.arange(tx_min, tx_max, grid_step),
                        np.arange(ty_min, ty_max, grid_step))

    # Interpolate power values for each point on the grid using Rbf
    rbf = Rbf(SunX, SunY, STOKE_V, function='linear')
    interp_power_STOKE_V = rbf(tx, ty)    


    # Define metadata for the solar map
    metadata = {
        'date-obs': f'{observation.year}-{observation.month}-{observation.day}T{observation.hour_start}:{observation.minute_start}:00',  # Adjust this to the correct observation date
        'crval1': 0,
        'crval2': 0,
        'cdelt1': grid_step,
        'cdelt2': grid_step,
        'cunit1': 'arcsec',
        'cunit2': 'arcsec',
        'ctype1': 'HPLN-TAN',
        'ctype2': 'HPLT-TAN',
        'crpix1': (tx_max - tx_min) / (2 * grid_step),
        'crpix2': (ty_max - ty_min) / (2 * grid_step),
        'waveunit': 'm',
        'wavelnth': 0.0262897 * u.m,
        'obsrvtry': 'Ventspils International Radio Astronomy Center',
        'detector': 'LNSP4',
        'dsun_ref': 149597870691,
        'dsun_obs': 151846026489 ,
        'rsun': 1573.89688496,
        'rsun_ref': 696000000 ,                           
        'hglt_obs': 0 * u.deg,
        'hgln_obs': 0 * u.deg,              
        
    }

    # Create a map using the interpolated power values and metadata STOKE I
    interpolated_map = sunpy.map.Map((interp_power_STOKE_I, metadata))
    # Plot the interpolated map using a heatmap with the 'hot' colormap
    plt.ioff()
    plt.rcParams['text.color'] = 'white'
    plt.rcParams['axes.labelcolor'] = 'white'
    plt.rcParams['xtick.color'] = 'white'
    plt.rcParams['ytick.color'] = 'white'   
    fig = plt.figure(figsize=(8, 8))
    fig.patch.set_facecolor('black')            
    ax = fig.add_subplot(projection=interpolated_map)           
    interpolated_map.plot(axes=ax,cmap='gist_heat')
    interpolated_map.draw_limb(axes=ax)
    interpolated_map.draw_grid(axes=ax)
    plt.colorbar(label='Power')
    plt.title(f'STOKE I | {band} {observation.year}-{observation.month:02d}-{observation.day:02d}T{observation.hour_start:02d}:{observation.minute_start:02d}')
    plt.xlabel('X (arcsec)')
    plt.ylabel('Y (arcsec)')
    

    # Calculate the radius in arcsec from the band in GHz to beam size
    beam_size = 1.22 * (3e8 / (float(band[:-3]) * 1e9)) / 32  # 32m is the diameter of the antenna
    beam_size_arcsec = (beam_size * (180 / np.pi) * 3600)  # Convert from radians to arcseconds 
    pixel_coords = interpolated_map.world_to_pixel(SkyCoord(900 * u.arcsec, -900 * u.arcsec, frame=interpolated_map.coordinate_frame))
    circle = plt.Circle((pixel_coords.x.value, pixel_coords.y.value), (beam_size_arcsec / grid_step)/2, 
                        color='white', fill=False, linestyle='--', transform=ax.transData)

    # Add a legend indicating the circle is the beam size
    legend_circle = plt.Line2D([0], [0], linestyle='--', color='white', label='Beam Size')
    ax.legend(handles=[legend_circle], loc='upper right', fontsize='small', frameon=False)
    # Add the circle to the axis
    ax.add_patch(circle)

    plt.grid(True)   
    name = f'{directory}/LNSP4-{directory}-STOKE_I-{band}.jpeg'
    plt.savefig(name, format='jpeg', dpi=300)     
    plt.close()

    # Create a map using the interpolated power values and metadata STOKE V
    interpolated_map = sunpy.map.Map((interp_power_STOKE_V, metadata))

    # Plot the interpolated map using a heatmap with the 'hot' colormap
    plt.ioff()
    plt.rcParams['text.color'] = 'white'
    plt.rcParams['axes.labelcolor'] = 'white'
    plt.rcParams['xtick.color'] = 'white'
    plt.rcParams['ytick.color'] = 'white'   
    fig = plt.figure(figsize=(8, 8))
    fig.patch.set_facecolor('black')            
    ax = fig.add_subplot(projection=interpolated_map)           
    interpolated_map.plot(axes=ax,cmap='nipy_spectral')
    interpolated_map.draw_limb(axes=ax)
    interpolated_map.draw_grid(axes=ax)         
    plt.colorbar(label='Power')
    plt.title(f'STOKE V | {band} {observation.year}-{observation.month:02d}-{observation.day:02d}T{observation.hour_start:02d}:{observation.minute_start:02d}')
    plt.xlabel('X (arcsec)')
    plt.ylabel('Y (arcsec)')

    # Calculate the radius in arcsec from the band in GHz to beam size
    beam_size = 1.22 * (3e8 / (float(band[:-3]) * 1e9)) / 32  # 32m is the diameter of the antenna
    beam_size_arcsec = (beam_size * (180 / np.pi) * 3600)  # Convert from radians to arcseconds 
    pixel_coords = interpolated_map.world_to_pixel(SkyCoord(900 * u.arcsec, -900 * u.arcsec, frame=interpolated_map.coordinate_frame))
    circle = plt.Circle((pixel_coords.x.value, pixel_coords.y.value), (beam_size_arcsec / grid_step)/2, 
                        color='white', fill=False, linestyle='--', transform=ax.transData)

    # Add a legend indicating the circle is the beam size
    legend_circle = plt.Line2D([0], [0], linestyle='--', color='white', label='Beam Size')
    ax.legend(handles=[legend_circle], loc='upper right', fontsize='small', frameon=False)
    # Add the circle to the axis
    ax.add_patch(circle)
    
    plt.grid(True)
    name = f'{directory}/LNSP4-{directory}-STOKE_V-{band}.jpeg'
    plt.savefig(name, format='jpeg', dpi=300)   
    plt.close()

Creating solar maps...
